In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0

In [31]:
!pip install ftfy regex tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00


In [32]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-j8ps1zk2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-j8ps1zk2
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369409 sha256=f5b6a057f085932adadac26264079e210b4b4bfee94e634f4e2e204485430f88
  Stored in directory: /tmp/pip-ephem-wheel-cache-cgrcaoby/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [1]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [2]:
from matplotlib import pyplot
from keras.datasets import cifar10
# load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [23]:
X=[]
y=[]

for i in X_train:
  X.append(i)

for i in X_test:
  X.append(i)

for i in y_train:
  y.append(i)

for i in y_test:
  y.append(i)      

In [4]:
import numpy as np

X=np.array(X)
y=np.array(y)

In [24]:
X=X[0:10000]
y=y[0:10000]

In [6]:
from PIL import Image
from matplotlib import cm

preprocessed=[preprocess(Image.fromarray(i)).unsqueeze(0).to(device) for i in X]

In [7]:
with torch.no_grad():
    encoded=[model.encode_image(i) for i in preprocessed]

In [12]:
with open('tt.npy', 'wb') as f:
    np.save(f, np.array(encoded),allow_pickle=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [14]:
with open('tt.npy', 'rb') as f:
    encoded = np.load(f, allow_pickle=True)

In [25]:
y=[i[0] for i in y]

In [16]:
shuffled_y=y

In [17]:
def shuffler (arr, n):
    # We will Start from the last element
    # and swap one by one.
    for i in range((n-1),0,-1):
         
        # Pick a random index from 0 to i
        j = random.randint(0,i)
         
        # Swap arr[i] with the element at random index
        arr[i],arr[j] = arr[j],arr[i]
    return arr

In [18]:
import random
shuffled_y=shuffler(shuffled_y,1000)

In [21]:
labels=list(set(y))

In [26]:
count_mislabeled=0
wrong_indices=[]

for i in range(0,len(y)):
    if y[i]!=shuffled_y[i]:
        count_mislabeled+=1
        wrong_indices.append(i)

In [93]:
count_mislabeled

892

In [28]:
import numpy as np
encoded=[np.array(encoded[i][0]) for i in range(0,len(encoded))]
encoded=np.array(encoded)

In [29]:
super=[]

for i in labels:
    X=[]
    for j in range(0,encoded.shape[0]):
        if shuffled_y[j]==i:
            X.append(np.array(encoded[j]))
    X=np.array(X)
    super.append(X)

In [120]:
from sklearn.ensemble import IsolationForest
from numpy import random, where

IF=IsolationForest(n_estimators=1000,contamination=float(0.0092))

values=[]

for i in super:
        pred = IF.fit_predict(i)
        anom_index = where(pred == -1)
        anom_index=list(anom_index)
        values.append(i[anom_index])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if sys.path[0] == "":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if sys.path[0] == "":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, w

In [121]:
anoms=[]
for i in range(0,len(values)):
    for j in range(0,len(values[i])):
        anoms.append(np.where(encoded==values[i][j])[0][0])

In [123]:
np.where(np.array(anoms)>=999)[0].shape

(8,)